# Code demonstrating basic non-linear reduced rank regression

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg
import torch

from janelia_core.ml.non_linear_rr_regression import NonLinearRRRegresion

## Setup ground truth model

In [3]:
d_in, d_out, d_latent, n_smps = 30000, 30000, 1, 1000

In [4]:
true_model = NonLinearRRRegresion(d_in, d_out, d_latent)
true_model.init_weights()

## Generate data

In [5]:
x = .1*torch.rand([n_smps, d_in])
y = true_model.generate(x)

## Create model we will fit to data

In [6]:
fitted_model = NonLinearRRRegresion(d_in, d_out, d_latent)
fitted_model.init_weights()
fitted_model.to('cuda')

KeyboardInterrupt: 

## Fit model to data

In [ ]:
fitted_model.fit(x,y, batch_size=1, max_its=100000)

## See fitted model results

In [ ]:
%matplotlib qt
fitted_model.standardize()
true_model.standardize()
NonLinearRRRegresion.compare_models(true_model, fitted_model, x, [0, 1])

In [ ]:
fitted_model.w0.device